In [61]:
from stmpy import Driver, Machine


class Quiz:
    def on_init(self):
        print("Init!")
        self.number_of_questions = 0
        self.question = "When is the weekend?"
        self.answering = None
        
    def send_question(self, question):
        print("Asking: {}".format(self.question))
        self.number_of_questions += 1
        self.question = question
        self.mqtt_client.publish("ttm4115/group07/client/question", self.question, qos=2)

    def receive_buzzer(self, answering):
        self.answering = answering
        self.mqtt_client.publish("ttm4115/group07/client/answering", self.answering, qos=2)
        print("Received buzz from: {}".format(self.answering))
    
    def handle_impatience_buzzer_early(self):
        self.mqtt_client.publish("ttm4115/group07/client/debug", f"Hold on, wait for a question!")
    
    def handle_impatience_buzzer_dupe(self):
        self.mqtt_client.publish("ttm4115/group07/client/debug", f"Hold on, {self.answering} is answering!")
    
    def handle_impatience_question_untimely(self):
        self.mqtt_client.publish("ttm4115/group07/server/debug","You already asked a question! Now waiting for anwers.")
    
    def handle_impatience_question_dupe(self):
        self.mqtt_client.publish("ttm4115/group07/server/debug","You already asked a question! Now waiting for anwers.")
    
    def internal_print(self, msg:str):
        print(msg)
        

s0 = {'name'  : 'quiz_master',
      'entry' : 'internal_print("quiz_master")',
      'buzzer': 'handle_impatience_buzzer_early()',
      'exit'  : 'start_timer("timer_0", 20000)',
      'exit'  : 'stop_timer("timer_0")'
     }

s1 = {'name'    : 'wait_for_buzzer',
      'entry'   : 'internal_print("wait_for_buzzer")',
      'question': 'handle_impatience_question_dupe()'
     }
s2 = {'name'  : 'handle_answer',
      'entry' : 'internal_print("handle_answer")',
      'entry' : 'start_timer("timer_1", 10000)',
      'buzzer': 'handle_impatience_buzzer_dupe()',
      'exit'  : 'stop_timer("timer_1")'
     }

# initial transition
t0 = {"source": "initial", "target": "quiz_master", "effect": "on_init"}

t1 = {
    "trigger": "question",
    "source": "quiz_master",
    "target": "wait_for_buzzer",
    "effect": "send_question(*)"
}

t2 = {
    "trigger": "timer_0",
    "source": "wait_for_buzzer",
    "target": "quiz_master",
}
t3 = {
    "trigger": "buzzer",
    "source": "wait_for_buzzer",
    "target": "handle_answer",
    "effect": "receive_buzzer(*)"
}
t4 = {
    "trigger": "timer_1",
    "source": "handle_answer",
    "target": "quiz_master"
} 

In [62]:
from threading import Thread

import paho.mqtt.client as mqtt


# The clients should subscribe to a channel as well as a  

class MQTT_Client_1:
    def __init__(self):
        self.count = 0
        self.client = mqtt.Client()
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message
        # channels to subscribe to
        self.client_channel = "ttm4115/group07/client/participants/#"
        self.server_channel = "ttm4115/group07/server/question"

    def on_connect(self, client, userdata, flags, rc):
        print("on_connect(): {}".format(mqtt.connack_string(rc)))

    def on_message(self, client, userdata, msg):
        print("on_message(): topic: {}".format(msg.topic))
        print("on_message(): payload: {}".format(msg.payload))
        
        
        if(msg.topic == self.server_channel):
            if(msg.retain):return 
            self.stm_driver.send("question", "quiz_machine", args=[msg.payload])
        
        elif(self.client_channel[:-1] in msg.topic):
            if(msg.retain):return
            name = str(msg.topic).split('/')[-1]
            print("HERE IS THE NAME YOU WANTED: ", name)
            self.stm_driver.send("buzzer", "quiz_machine", args=[name])
        
            
    def start(self, broker, port):

        print("Connecting to {}:{}".format(broker, port))
        self.client.connect(broker, port)

        self.client.subscribe(self.server_channel)
        self.client.subscribe(self.client_channel)

        try:
            # line below should not have the () after the function!
            thread = Thread(target=self.client.loop_forever)
            thread.start()
        except KeyboardInterrupt:
            print("Interrupted")
            self.client.disconnect()


In [63]:
# broker, port = 'iot.eclipse.org', 1883
broker, port = "mqtt.item.ntnu.no", 1883

quiz = Quiz()
quiz_machine = Machine(transitions=[t0, t1, t2, t3, t4], obj=quiz, name="quiz_machine", states=[s0,s1,s2])
quiz.stm = quiz_machine

driver = Driver()
driver.add_machine(quiz_machine)

myclient = MQTT_Client_1()
quiz.mqtt_client = myclient.client
myclient.stm_driver = driver

driver.start()
myclient.start(broker, port)

Init!Connecting to mqtt.item.ntnu.no:1883
quiz_master

on_connect(): Connection Accepted.
on_message(): topic: ttm4115/group07/server/question
on_message(): payload: b'Retained Question'
on_message(): topic: ttm4115/group07/client/participants/Hallvard
on_message(): payload: b'Retained Buzzer'
